[![Binder](img/badge-binder.svg)](https://mybinder.org/v2/gh/nhirschey/teaching/gh-pages?filepath=assignment-volatility-timing.ipynb)&emsp;
[![Script](img/badge-script.svg)](/Teaching//assignment-volatility-timing.fsx)&emsp;
[![Notebook](img/badge-notebook.svg)](/Teaching//assignment-volatility-timing.ipynb)

Group Name:

Student Name | Student Number
--- | ---
**1:Bernardo Manarte** |55810 &#32;
**2:Leonor Rodrigues** |48778 &#32;
**3:Marouan Kamoun** |53833 &#32;
**4:Rodrigo Simões** |53154 &#32;
**5:Vasco Calxito** |53960 &#32;


This is an assignment. You should work in groups. Please write your group and group member names above. You will find sections labeled **Task** asking you to do each piece of analysis. Please make sure that you complete all of these tasks. I included some tests to help you see if you are calculating the solution correctly, but if you cannot get the test to pass submit your best attempt and you may recieve partial credit.

All work that you submit should be your own. Make use of the course resources and example code on the course website. It should be possible to complete all the requested tasks using information given below or somewhere on the course website.



In [1]:
#r "nuget:FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: Plotly.NET, 3.*"


Installed Packages FSharp.Data, 5.0.2 FSharp.Stats, 0.4.11 Plotly.NET, 3.0.1

In [2]:
#r "nuget: Plotly.NET.Interactive, 3.*"


Installed Packages Plotly.NET.Interactive, 3.0.2

Loading extensions from `/Users/rodrigosimoes/.nuget/packages/plotly.net.interactive/3.0.2/interactive-extensions/dotnet/Plotly.NET.Interactive.dll`

In [3]:
open System
open FSharp.Data
open Plotly.NET
open FSharp.Stats


for testing.



In [4]:
#r "nuget: FsUnit.Xunit"
#r "nuget: xunit, 2.*"
open Xunit
open FsUnit.Xunit
open FsUnitTyped


Installed Packages FsUnit.Xunit, 5.2.0 xunit, 2.4.2

In [5]:
// Set dotnet interactive formatter to plaintext
Formatter.Register(fun (x:obj) (writer: TextWriter) -> fprintfn writer "%120A" x )
Formatter.SetPreferredMimeTypesFor(typeof<obj>, "text/plain")
// Make plotly graphs work with interactive plaintext formatter
Formatter.SetPreferredMimeTypesFor(typeof<GenericChart.GenericChart>,"text/html")


## Load Data

We get the Fama-French 3-Factor asset pricing model data.



In [6]:
#r "nuget: NovaSBE.Finance"

open NovaSBE.Finance
open NovaSBE.Finance.French

let ff3 = 
    French.getFF3 Frequency.Daily
    |> Seq.toList
    |> List.filter (fun x -> x.Date < DateTime(2023,3,1))

let annualizeDailyStdDev dailyStdDev = sqrt(252.0) * dailyStdDev


Microsoft.DotNet.Interactive.InstallPackagesMessage


As an example, I'll first calculate the standard deviation of the `MktRf` factor and assign it to a value named `stdDevMkt`.



In [7]:
let stdDevMkt =
    ff3
    |> Seq.stDevBy (fun x -> x.MktRf)


The following test will pass if I calculate it correctly.



In [8]:
// Test.
stdDevMkt 
|> should (equalWithin 0.005) 0.01


The test following test will fail if I calculate it incorrectly. In this failing example I report an annualized standard deviation instead of a daily standard deviation.



In [30]:
let stdDevMktFAIL =
    ff3
    |> Seq.stDevBy (fun x -> x.MktRf)
    |> annualizeDailyStdDev

// Test
if false then // make this `if true` to run the test.
    stdDevMktFAIL
    |> should (equalWithin 0.005) 0.01


## Start of the assignment

> **Task:** Calculate the standard deviation of the `Hml` factor's daily returns. Assign it to a value named `stdDevHml`.
> 

Write your solution in the cell below.



In [10]:
// Solution here

let stdDevHml =
    ff3
    |> Seq.stDevBy (fun x -> x.Hml)
// Test
if true then
    stdDevHml 
    |> should (equalWithin 0.005) 0.006 
    printfn "Test passed"


Test passed


> **Task:** Calculate the annualized standard deviation of the `Hml` factor's daily returns. Assign it to a value named `stdDevHmlAnnualized`.
> 

Write your solution in the cell below.



In [11]:
// Solution here
let stdDevHmlAnnualized =
    ff3
    |> Seq.stDevBy (fun x -> x.Hml)
    |> annualizeDailyStdDev
// Test
if true then
    stdDevHmlAnnualized 
    |> should (equalWithin 0.005) 0.098
    printfn "Test passed"

Test passed


> **Task:** Assign the daily returns of the `Hml` factor to a value named `hml` that is a `list` of `ReturnObs`.
> 



In [12]:
type ReturnObs = 
    { 
        Name: string
        Date: DateTime 
        Return: float 
    }


Write your solution in the cell below.



In [13]:
// Solution here
let hml : ReturnObs list =
    ff3
    |> List.map (fun x -> 
        { Name = "hml"; Date = x.Date; Return = x.Hml })
        
// Tests
hml[..1] |> should be ofExactType<list<ReturnObs>>

hml[0].Name |> shouldEqual "hml"

hml |> shouldHaveLength 25419

hml
|> List.averageBy (fun x -> x.Return)
|> should (equalWithin 0.0001) 0.00015


> **Task:** Calculate the daily returns of the `Hml` factor with a constant `2x` leverage applied to the portfolio every day. Assign it to a value named `hml2x` that is a `list` of `ReturnObs`.
> 

Write your solution in the cell below.



In [34]:
// Solution here
let hml2x : ReturnObs list =
    ff3
    |> List.map (fun x -> 
        { Name = "hml2x"; Date = x.Date; Return = 2.0 * x.Hml })


// Tests
hml2x[..1] |> should be ofExactType<list<ReturnObs>>

hml2x[0].Name |> shouldEqual "hml2x"

hml2x |> shouldHaveLength 25419

hml2x
|> List.averageBy (fun x -> x.Return)
|> should (equalWithin 0.0001) 0.0003


> **Task:** Calculate the average annualized daily return of the `Hml` factor and assign it to a value name `hmlAvg`.
> 

Write your solution in the cell below.



In [39]:
// solution here
let hmlAvg = 
    ff3 
    |> List.map (fun x -> x.Hml) 
    |> List.average
    |> (*) 252.0

// tests
hmlAvg |> should (equalWithin 0.005) 0.04


> **Task:** Calculate the average annualized daily return of the 2x leveraged `Hml` factor and assign it to a value name `hml2xAvg`.
> 

Write your solution in the cell below.



In [40]:
// solution here
let hml2xAvg = 
    hml2x 
    |> List.map (fun x -> x.Return) 
    |> List.average 
    |> (*) 252.0


// tests
hml2xAvg |> should (equalWithin 0.005) 0.075


Here is some code that you will use for the next task.



In [17]:
type GrowthObs =
    { 
        Name: string
        Date: DateTime
        Growth: float
    }

let cumulativeGrowth (xs: list<ReturnObs>) =
    let sorted = xs |> List.sortBy (fun x -> x.Date)
    let calcGrowth (prior: GrowthObs) (current: ReturnObs) =
        { Name = current.Name 
          Date = current.Date
          Growth = prior.Growth * (1.0 + current.Return) }        
    match sorted with
    | [] -> []
    | h::t ->
        let firstOb = 
            { Name = h.Name 
              Date = h.Date
              Growth = 1.0 + h.Return }
        (firstOb, t) ||> List.scan calcGrowth


> **Task:** Calculate the cumulative growth of $1 invested in HML at the start of the sample. Assign it to a value named `hmlGrowth` that is a `list` of `GrowthObs`.
> 

Write your solution in the cell below.



In [18]:
// Solution here

// Tests
hmlGrowth[..1] |> should be ofExactType<list<GrowthObs>>

hmlGrowth
|> List.map (fun x -> x.Growth)
|> List.last
|> should (equalWithin 1.0) 29.0


Error: input.fsx (4,1)-(4,10) typecheck error The value or constructor 'hmlGrowth' is not defined.
input.fsx (6,1)-(6,10) typecheck error The value or constructor 'hmlGrowth' is not defined.

> **Task:** Calculate the cumulative growth of $1 invested in 2x levered HML at the start of the sample. Assign it to a value named `hml2xGrowth` that is a `list` of `GrowthObs`.
> 

Write your solution in the cell below.



In [19]:
// Solution here

// Tests
hml2xGrowth[..1] |> should be ofExactType<list<GrowthObs>>

hml2xGrowth
|> List.map (fun x -> x.Growth)
|> List.last
|> should (equalWithin 1.0) 332.0


Error: input.fsx (4,1)-(4,12) typecheck error The value or constructor 'hml2xGrowth' is not defined.
input.fsx (6,1)-(6,12) typecheck error The value or constructor 'hml2xGrowth' is not defined.

Here is an example of a plot of the cumulative growth of $1 invested in the market.



In [20]:
let mkt = 
    [ for x in ff3 do
        { Name = "market"
          Date = x.Date
          Return = x.MktRf } ]

let marketGrowthChart = 
    mkt
    |> cumulativeGrowth
    |> List.map (fun x -> x.Date, x.Growth)
    |> Chart.Line
    |> Chart.withYAxisStyle (AxisType = StyleParam.AxisType.Log)


In [21]:
marketGrowthChart


<!-- Plotly chart will be drawn inside this DIV -->

In [22]:
Warning: Output, it-value and value references require --eval


Error: input.fsx (1,8)-(1,9) parse error Unexpected symbol ':' in interaction. Expected incomplete structured construct at or before this point, ';', ';;' or other token.
input.fsx (1,8)-(1,9) interactive error parse error

> **Task:** Plot the cumulative growth of $1 invested in HML and $1 invested in the Market as a combined line chart using a log scale for the y-axis.
> 

Write your solution in the cell below.



In [23]:
// Solution here


> **Task:** Apply a constant levarage to the HML and market factors for the full sample such that the daily returns have a full-sample annualized standard deviation equal to 10%. Remember, **constant leverage** means that leverage is exactly the same every day. Assign the results to values named `hml10` and `mkt10` that are lists of `ReturnObs`.
> 

Write your solution in the cell below.



In [24]:
// Solution here


// Tests
hml10[..1] |> should be ofExactType<list<ReturnObs>>
mkt10[..1] |> should be ofExactType<list<ReturnObs>>

hml10 
|> stDevBy (fun x -> x.Return) 
|> annualizeDailyStdDev
|> should (equalWithin 1e-6) 0.1

mkt10
|> stDevBy (fun x -> x.Return) 
|> annualizeDailyStdDev
|> should (equalWithin 1e-6) 0.1


Error: input.fsx (5,1)-(5,6) typecheck error The value or constructor 'hml10' is not defined. Maybe you want one of the following:
   hml
input.fsx (6,1)-(6,6) typecheck error The value or constructor 'mkt10' is not defined. Maybe you want one of the following:
   mkt
input.fsx (8,1)-(8,6) typecheck error The value or constructor 'hml10' is not defined. Maybe you want one of the following:
   hml
input.fsx (13,1)-(13,6) typecheck error The value or constructor 'mkt10' is not defined. Maybe you want one of the following:
   mkt

> **Task:** Plot the cumulative growth of $1 invested in `hml10` and `mkt10` as a combined line chart using a log scale for the y-axis.
> 

Write your solution in the cell below.



In [25]:
// Solution here


> **Task:** Explain how to intepret the dramatic difference between the two plots that you have just created. Why is the plot of $1 invested in unlevered HML and Market factors so different from the plot of $1 invested in the versions that are levered to have a 10% annualized standard deviation?
> 

Write your answer here.

> **Task:** Construct a volatility managed HML portfolio that puts a weight of $1/\hat{\sigma^2_t}$ on HML,
where $\hat{\sigma_t}$ is estimated as the **annualized** standard deviation of returns over the prior 22 days from t-1 to t-22.
> 

The result should be a `list<ReturnOb>` named `hmlVolManaged`.

Write your solution in the cell below.



In [26]:
// Solution here


// Tests
hmlVolManaged |> should be ofExactType<list<ReturnObs>>

let hmlAnnualizedReturn =
    let dailyRet =
        hmlVolManaged
        |> List.averageBy (fun x -> x.Return)
    252.0 * dailyRet
hmlAnnualizedReturn |> should (equalWithin 0.02) 8.18


Error: input.fsx (5,1)-(5,14) typecheck error The value or constructor 'hmlVolManaged' is not defined.
input.fsx (9,9)-(9,22) typecheck error The value or constructor 'hmlVolManaged' is not defined.